In [1]:
import pandas as pd
import numpy as np
import json
import os

from pathlib import Path
from tqdm import tqdm

In [2]:
raw_tables_dir='../wikitables/files/wikitables_raw/'
tables_output_dir='wikipages_expanded_dataset_strubert_format/tables/'
queries_output_dir='wikipages_expanded_dataset_strubert_format/queries/'
input_tables_dir='wikipages_expanded_dataset/tables/'

In [3]:
input_tables=set(os.listdir(input_tables_dir))

# Functions

In [5]:
def remove_non_cell_value_fields(table_dict):
    '''
    Given the dictionary of a table (as presented in the raw json files) replace all non-data values
    with the empty "" string
    '''
    table_dict['title'] = [""] * len(table_dict['title'])
    table_dict['pgTitle'] = ""
    table_dict['secondTitle'] = ""
    table_dict['caption'] = ""

    return table_dict

def select_data_rows(table_dict, selected_row_ids):
    '''
    Given the dictionary of a table (as presented in the raw json files) select only the specified `selected_row_ids`
    from its data field.
    '''
    table_dict['data'] = [table_dict['data'][i] for i in selected_row_ids]
    return table_dict

# Construct Tables

In [13]:
# Loop over all raw tables and select the ones in the `input_tables` set.
# The selected tables are saved in the output_dir
for filename in tqdm(sorted(os.listdir(raw_tables_dir))):
    with open(raw_tables_dir+filename, 'r') as f:
        data = json.load(f)

    filtered_dict = dict()

    for table in data:
        if table +'.json' in input_tables:
            table_dict = data[table]
            # Set non-cell value fields to the empty "" string
            table_dict = remove_non_cell_value_fields(table_dict)
            filtered_dict[table] = table_dict
    
    # Check if the filtered dict is not empty and save it to the `tables_output_dir`
    if len(filtered_dict) > 0:
        with open(tables_output_dir + filename, 'w') as fp:
            json.dump(filtered_dict, fp,  indent=2)

100%|██████████| 1653/1653 [02:54<00:00,  9.45it/s]


In [14]:
# Sanity check. Check how many unique tables there are in total under the `tables_output_dir`
unique_tables = set()
for filename in tqdm(os.listdir(tables_output_dir)):
    with open(tables_output_dir+filename, 'r') as f:
        data = json.load(f)
    for table_name in data:
        unique_tables.add(table_name)
print('There are in total', len(unique_tables), 'unique tables under the', tables_output_dir, 'directory.')

100%|██████████| 1650/1650 [00:16<00:00, 98.49it/s] 

There are in total 238038 unique tables under the wikipages_expanded_dataset_strubert_format/tables/ directory.


# Construct Queries

In [9]:
queries_df = pd.read_pickle('../../queries/wikipages/query_dataframes/expanded_wikipages/filtered_queries/minTupleWidth_all_tuplesPerQuery_all.pickle')
queries_df

,wikipage,wikipage_id,num_tables,tables,num_entities,tuple_width,num_tuples,selected_table,selected_row_ids,categories_relevant_wikipages,categories_relevant_tables,navigation_links_relevant_wikipages,navigation_links_relevant_tables,categories_expansion_ratio,navigation_links_expansion_ratio,avg_query_containment
32,https://en.wikipedia.org/wiki/Andre_Norton_Award,32,1,[table-0001-242.json],[80],4.0,11.0,table-0001-242.json,"[4, 6, 7, 10, 13, 14, 28, 30, 31, 33, 36]",51.0,58.0,NaN,NaN,58.0,NaN,0.036120
46,https://en.wikipedia.org/wiki/President_of_Ind...,46,1,[table-0001-319.json],[20],3.0,10.0,table-0001-319.json,"[0, 3, 4, 5, 6, 7, 8, 12, 14, 18]",7.0,7.0,NaN,NaN,7.0,NaN,0.271930
66,https://en.wikipedia.org/wiki/Charlotte_Bobcat...,66,1,[table-0001-460.json],[108],3.0,28.0,table-0001-460.json,"[0, 8, 9, 15, 19, 21, 26, 28, 31, 34, 38, 39, ...",27.0,122.0,NaN,NaN,122.0,NaN,0.026968
68,https://en.wikipedia.org/wiki/List_of_organism...,68,1,[table-0001-469.json],[355],3.0,14.0,table-0001-469.json,"[2, 44, 45, 53, 80, 107, 206, 208, 213, 219, 2...",16.0,37.0,NaN,NaN,37.0,NaN,0.002137
97,https://en.wikipedia.org/wiki/1982_NCAA_Women'...,97,2,"[table-0001-64.json, table-0001-65.json]","[12, 58]",4.0,13.0,table-0001-65.json,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 13, 14, 15]",34.0,39.0,NaN,NaN,19.5,NaN,0.210526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244164,https://en.wikipedia.org/wiki/2004_LPGA_Tour,244164,2,"[table-1653-355.json, table-1653-356.json]","[64, 15]",3.0,10.0,table-1653-355.json,"[4, 6, 7, 8, 10, 16, 17, 24, 26, 30]",68.0,81.0,NaN,NaN,40.5,NaN,0.237500
244174,https://en.wikipedia.org/wiki/1998_NCAA_Women'...,244174,1,[table-1653-409.json],[56],4.0,11.0,table-1653-409.json,"[0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 15]",40.0,48.0,NaN,NaN,48.0,NaN,0.206687
244209,https://en.wikipedia.org/wiki/1931_Italian_Gra...,244209,1,[table-1653-615.json],[26],3.0,13.0,table-1653-615.json,"[0, 1, 2, 3, 4, 11, 15, 19, 21, 22, 23, 24, 27]",88.0,129.0,NaN,NaN,129.0,NaN,0.075247
244229,https://en.wikipedia.org/wiki/List_of_World_Aq...,244229,16,"[table-1653-718.json, table-1653-720.json, tab...","[30, 44, 45, 40, 40, 44, 41, 24, 21, 47, 25, 4...",4.0,15.0,table-1653-730.json,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",44.0,240.0,NaN,NaN,15.0,NaN,0.131381


In [10]:
query_tables = set(queries_df['selected_table'])

num_tuples_per_query_list = [1,2,5,10]

# Loop over all raw tables and select the ones in the `query_tables` set.
# The selected tables are saved in the output_dir
for filename in tqdm(sorted(os.listdir(raw_tables_dir))):
    with open(raw_tables_dir+filename, 'r') as f:
        data = json.load(f)

    dict_of_filtered_dict = dict()
    dict_of_filtered_dict['full'] = dict()
    dict_of_filtered_dict['1_rows_per_query'] = dict()
    dict_of_filtered_dict['2_rows_per_query'] = dict()
    dict_of_filtered_dict['5_rows_per_query'] = dict()
    dict_of_filtered_dict['10_rows_per_query'] = dict()

    for table in data:
        if table +'.json' in query_tables:
            table_dict = data[table]
            # Set non-cell value fields to the empty "" string
            table_dict = remove_non_cell_value_fields(table_dict)

            # Get the selected row IDS
            selected_row_ids = queries_df[queries_df['selected_table']==table+'.json']['selected_row_ids'].tolist()[0]
            full_table_dict = select_data_rows(table_dict, selected_row_ids)
            dict_of_filtered_dict['full'][table] = full_table_dict

            # Update the table_dict by choosing variable sized tuples per query
            for tuple_size in num_tuples_per_query_list:
                tmp_table_dict = full_table_dict.copy()
                tmp_table_dict['data'] = tmp_table_dict['data'][:tuple_size]
                dict_of_filtered_dict[str(tuple_size) + '_rows_per_query'][table] = tmp_table_dict

  
    # Check if the filtered dict is not empty and save it to the `queries_output_dir`
    if len(dict_of_filtered_dict['full']) > 0:
        with open(queries_output_dir + 'full/' + filename, 'w') as fp:
            json.dump(dict_of_filtered_dict['full'], fp,  indent=2)

        # Save the variable number of tuples per query files
        for tuple_size in num_tuples_per_query_list:
            with open(queries_output_dir + str(tuple_size) + '_rows_per_query/' + filename, 'w') as fp:
                json.dump(dict_of_filtered_dict[str(tuple_size) + '_rows_per_query'], fp,  indent=2)

100%|██████████| 1653/1653 [02:40<00:00, 10.31it/s]


In [11]:
# Construct the query.txt file
query_txt_output_dir='wikipages_expanded_dataset_strubert_format/queries/'
query_file_dict={'query_id': [], 'query_table': []}
for idx, row in queries_df.iterrows():
    query_file_dict['query_id'].append(row['wikipage_id'])
    query_file_dict['query_table'].append(os.path.splitext(row['selected_table'])[0])

query_file_df = pd.DataFrame.from_dict(query_file_dict)
query_file_df.to_csv(query_txt_output_dir+'query.txt', index=False, header=False, sep='\t')

## Sampled Queries

In [13]:
# Perform the same procedure but only over the sampled queries (i.e., copy files only for the sampled queries)
sampled_queries_output_dir = 'wikipages_expanded_dataset_strubert_format/sampled_queries/'
sampled_queries_df = pd.read_pickle('../../queries/wikipages/query_dataframes/expanded_wikipages/sampled_queries/sampled_queries.pickle')
query_tables = set(sampled_queries_df['selected_table'])

num_tuples_per_query_list = [1,2,5,10]

# Loop over all raw tables and select the ones in the `query_tables` set.
# The selected tables are saved in the output_dir
for filename in tqdm(sorted(os.listdir(raw_tables_dir))):
    with open(raw_tables_dir+filename, 'r') as f:
        data = json.load(f)

    dict_of_filtered_dict = dict()
    dict_of_filtered_dict['full'] = dict()
    dict_of_filtered_dict['1_rows_per_query'] = dict()
    dict_of_filtered_dict['2_rows_per_query'] = dict()
    dict_of_filtered_dict['5_rows_per_query'] = dict()
    dict_of_filtered_dict['10_rows_per_query'] = dict()

    for table in data:
        if table +'.json' in query_tables:
            table_dict = data[table]
            # Set non-cell value fields to the empty "" string
            table_dict = remove_non_cell_value_fields(table_dict)

            # Get the selected row IDS
            selected_row_ids = sampled_queries_df[sampled_queries_df['selected_table']==table+'.json']['selected_row_ids'].tolist()[0]
            full_table_dict = select_data_rows(table_dict, selected_row_ids)
            dict_of_filtered_dict['full'][table] = full_table_dict

            # Update the table_dict by choosing variable sized tuples per query
            for tuple_size in num_tuples_per_query_list:
                tmp_table_dict = full_table_dict.copy()
                tmp_table_dict['data'] = tmp_table_dict['data'][:tuple_size]
                dict_of_filtered_dict[str(tuple_size) + '_rows_per_query'][table] = tmp_table_dict

  
    # Check if the filtered dict is not empty and save it to the `sampled_queries_output_dir`
    if len(dict_of_filtered_dict['full']) > 0:
        with open(sampled_queries_output_dir + 'full/' + filename, 'w') as fp:
            json.dump(dict_of_filtered_dict['full'], fp,  indent=2)

        # Save the variable number of tuples per query files
        for tuple_size in num_tuples_per_query_list:
            with open(sampled_queries_output_dir + str(tuple_size) + '_rows_per_query/' + filename, 'w') as fp:
                json.dump(dict_of_filtered_dict[str(tuple_size) + '_rows_per_query'], fp,  indent=2)

100%|██████████| 1653/1653 [02:28<00:00, 11.17it/s]


In [14]:
# Construct the query.txt file
query_txt_output_dir='wikipages_expanded_dataset_strubert_format/sampled_queries/'
query_file_dict={'query_id': [], 'query_table': []}
for idx, row in sampled_queries_df.iterrows():
    query_file_dict['query_id'].append(row['wikipage_id'])
    query_file_dict['query_table'].append(os.path.splitext(row['selected_table'])[0])

query_file_df = pd.DataFrame.from_dict(query_file_dict)
query_file_df.to_csv(query_txt_output_dir+'query.txt', index=False, header=False, sep='\t')

# Construct Ground Truth

In [15]:
def get_relevant_wikipages(wikipage_id, relevance_scores_dir):
    '''
    Given a wikipage_id return a dictionary keyed by the relevant wikipages and their relevance scores
    '''
    with open(relevance_scores_dir + str(wikipage_id) + '.json') as fp:
        relevant_wikipages_dict = json.load(fp)
    return relevant_wikipages_dict

def get_relevant_wikitables(relevant_wikipages, df):
    '''
    Given a dictionary of the relevant wikipages and their relevance scores return a dictionary
    of the relevant wikitables and their relevance scores
    '''
    relevant_wikitables_dict = dict()
    for wikipage in relevant_wikipages:
        wikipage_link = 'https://en.wikipedia.org/wiki/' + wikipage
        if wikipage_link in df['wikipage'].values:
            wikitables = df[df['wikipage']==wikipage_link]['tables'].values[0]
            for wikitable in wikitables:
                relevant_wikitables_dict[wikitable] = relevant_wikipages[wikipage]
  
    return relevant_wikitables_dict

def get_groundtruth_df(query_id, relevant_wikitables_dict, search_space_tables):
    '''
    Returns a dataframe of the groundtruth relevance for the specified `query_id` in the format
    used by StruBERT
    '''

    df_dict = {'query_id': [], 'dummy_var': [], 'table': [], 'relevance_score': []}
    for table in relevant_wikitables_dict:
        df_dict['query_id'].append(query_id)
        df_dict['dummy_var'].append(0)
        df_dict['table'].append(os.path.splitext(table)[0])
        df_dict['relevance_score'].append(relevant_wikitables_dict[table])

    # relevant_tables = set([os.path.splitext(table)[0] for table in relevant_wikitables_dict.keys()])
    # non_relevant_tables = search_space_tables - relevant_tables
    # for table in non_relevant_tables:
    #     df_dict['query_id'].append(query_id)
    #     df_dict['dummy_var'].append(0)
    #     df_dict['table'].append(table)
    #     df_dict['relevance_score'].append(0)

    gt_df = pd.DataFrame.from_dict(df_dict)
    return gt_df

In [6]:
df = pd.read_pickle('wikipages_expanded_dataset/wikipages_df.pickle')
queries_df = pd.read_pickle('../../queries/wikipages/query_dataframes/expanded_wikipages/filtered_queries/minTupleWidth_all_tuplesPerQuery_all.pickle')
categories_relevance_scores_dir = '../../queries/wikipages/groundtruth_generation/wikipage_relevance_scores/wikipages_expanded_dataset/jaccard_categories_new/'
groundtruth_output_dir='wikipages_expanded_dataset_strubert_format/groundtruth_query_relevance/only_relevant/'

In [7]:
# The set of tables in our search space
search_space_tables = set()
for filename in tqdm(os.listdir(tables_output_dir)):
    with open(tables_output_dir+filename, 'r') as f:
        data = json.load(f)
    for table_name in data:
        search_space_tables.add(table_name)
print("There are", len(search_space_tables), 'tables in our search space.')

100%|██████████| 1650/1650 [00:15<00:00, 104.33it/s]

There are 238038 tables in our search space.


In [ ]:
for idx, row in tqdm(queries_df.iterrows(), total=queries_df.shape[0]):
    wikipage_id=row['wikipage_id']
    query_table=os.path.splitext(row['selected_table'])[0]
    relevant_wikipages_dict = get_relevant_wikipages(wikipage_id=wikipage_id, relevance_scores_dir=categories_relevance_scores_dir)
    relevant_wikitables_dict = get_relevant_wikitables(relevant_wikipages=relevant_wikipages_dict, df=df)
    
    gt_df = get_groundtruth_df(query_id=wikipage_id, relevant_wikitables_dict=relevant_wikitables_dict, search_space_tables=search_space_tables)

    # Save the gt_df into a .tsv file
    gt_df.to_csv(groundtruth_output_dir+query_table+'.tsv', index=False, header=False, sep='\t')

## Sampled Queries

In [25]:
import shutil

df = pd.read_pickle('wikipages_expanded_dataset/wikipages_df.pickle')
sampled_queries_df = pd.read_pickle('../../queries/wikipages/query_dataframes/expanded_wikipages/sampled_queries/sampled_queries.pickle')
groundtruth_output_dir='wikipages_expanded_dataset_strubert_format/groundtruth_query_relevance/'
sampled_queries_groundtruth_output_dir='wikipages_expanded_dataset_strubert_format/sampled_queries_groundtruth_query_relevance/'

In [26]:
# Copy the .tsv files for each query table in the sampled_queries_df
for idx, row in tqdm(sampled_queries_df.iterrows(), total=sampled_queries_df.shape[0]):
    table_name = os.path.splitext(row['selected_table'])[0]

    # Copy full groundtruth
    shutil.copy(groundtruth_output_dir+'full/'+table_name+'.tsv', sampled_queries_groundtruth_output_dir+'full/')

    # Copy only relevant groundtruth
    shutil.copy(groundtruth_output_dir+'only_relevant/'+table_name+'.tsv', sampled_queries_groundtruth_output_dir+'only_relevant/')

100%|██████████| 120/120 [00:24<00:00,  4.97it/s]


In [43]:
# Create a shortened groundtruth for the sampled queries. The shortened groundtruth is composed by selecting all non-zero relevant tables for a query
# and adding randomly N non-relevant tables where N is equal tot he number of non-zero relevant tables
for filename in tqdm(sorted(os.listdir(sampled_queries_groundtruth_output_dir+'full/'))):
    full_gt_df = pd.read_csv(sampled_queries_groundtruth_output_dir + 'full/' + filename, sep='\t', names=['query_id', 'dummy_var', 'table', 'relevance_score'])

    short_gt_df = full_gt_df[full_gt_df['relevance_score'] > 0]
    
    # Randomly select len(short_gt_df) zero relevance rows from full_gt_df
    zero_relevance_rows_df = full_gt_df[full_gt_df['relevance_score'] == 0]
    sampled_rows = zero_relevance_rows_df.sample(n=len(short_gt_df), random_state=1)

    # Combine the short_gt_df with the sampled_rows    
    short_gt_df = pd.concat([short_gt_df, sampled_rows])

    # Save the dataframe
    short_gt_df.to_csv(sampled_queries_groundtruth_output_dir + 'shortened_gt/' + filename, index=False, header=False, sep='\t')

100%|██████████| 120/120 [00:33<00:00,  3.59it/s]


### Create k-fold cross validation splits for the sampled queries

In [63]:
from sklearn.model_selection import KFold
shortened_gt_dir = 'wikipages_expanded_dataset_strubert_format/sampled_queries_groundtruth_query_relevance/shortened_gt/'
k_folds_output_dir = 'wikipages_expanded_dataset_strubert_format/sampled_queries_groundtruth_query_relevance/k_folds/'

# Dictionaries keyed by the fold number and mapping to the list of dataframes (one for each filename)
train_dfs_dict = {1: [], 2: [], 3: [], 4: [], 5: []}
test_dfs_dict = {1: [], 2: [], 3: [], 4: [], 5: []}

# Loop over each file in the `shortened_gt_dir` and construct 
for filename in tqdm(sorted(os.listdir(shortened_gt_dir))):
    gt_df = pd.read_csv(shortened_gt_dir + filename, sep='\t', names=['query_id', 'dummy_var', 'table', 'relevance_score'])

    non_zero_rel_df = gt_df[gt_df['relevance_score']>0] 
    zero_rel_df = gt_df[gt_df['relevance_score']==0] 

    train_non_zero_rel_splits=[]
    test_non_zero_rel_splits=[]

    train_zero_rel_splits=[]
    test_zero_rel_splits=[]
    
    kf = KFold(n_splits = 5, shuffle = True, random_state = 1)   
    # Folds over the non-zero relevance tables
    for train, test in kf.split(non_zero_rel_df):
        train_non_zero_rel_splits.append(non_zero_rel_df.iloc[train])
        test_non_zero_rel_splits.append(non_zero_rel_df.iloc[test])

    # Folds over zero-relevance tables
    for train, test in kf.split(zero_rel_df):
        train_zero_rel_splits.append(zero_rel_df.iloc[train])
        test_zero_rel_splits.append(zero_rel_df.iloc[test])

    # Concatenate the zero and non-zero relevance test and train splits into a single dataframe
    for i in range(len(train_non_zero_rel_splits)):
        train_dfs_dict[i+1].append(pd.concat([train_non_zero_rel_splits[i], train_zero_rel_splits[i]]))
        test_dfs_dict[i+1].append(pd.concat([test_non_zero_rel_splits[i], test_zero_rel_splits[i]]))

# Save the folds by concatenating the list of dataframes
for i in range(1, 6):
    df_train = pd.concat(train_dfs_dict[i])
    df_test = pd.concat(test_dfs_dict[i])

    df_train.to_csv(k_folds_output_dir + 'train_fold_' + str(i) + '.tsv', index=False, header=False, sep='\t')
    df_test.to_csv(k_folds_output_dir + 'test_fold_' + str(i) + '.tsv', index=False, header=False, sep='\t')


100%|██████████| 120/120 [00:00<00:00, 120.24it/s]
